<a href="https://colab.research.google.com/github/Pavithra2625/News_Research_Tool/blob/main/News_Research_Tool_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio langchain langchain-community langchain-groq faiss-cpu sentence-transformers unstructured requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━

In [3]:
import os
import pickle
import time
import gradio as gr

from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
userdata.get('groq_API')


# 1. Set Groq API Key
os.environ["GROQ_API_KEY"] = "groq_API"

# 2. Initialize LLM (Groq)
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-70b-8192",  # Groq supported model
    temperature=0.7
)

file_path = "faiss_store_groq.pkl"
vectorstore = None

# 3. Function to process URLs and build FAISS index
def process_urls(url1, url2, url3):
    urls = [u for u in [url1, url2, url3] if u.strip()]
    if not urls:
        return "Please provide at least one valid URL."

    loader = UnstructuredURLLoader(urls=urls)
    try:
        data = loader.load()
    except Exception as e:
        return f"Error loading URLs: {str(e)}"

    if not data:
        return "No content fetched from the provided URLs."

    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
    docs = text_splitter.split_documents(data)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore_groq = FAISS.from_documents(docs, embeddings)

    with open(file_path, "wb") as f:
        pickle.dump(vectorstore_groq, f)

    return "Processing completed! You can now ask questions."

def answer_question(question):
    if not os.path.exists(file_path):
        return "Please process URLs first!", ""

    with open(file_path, "rb") as f:
        vectorstore = pickle.load(f)

    chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

    try:
        result = chain({"question": question}, return_only_outputs=True)
    except Exception as e:
        return f"Error generating answer: {str(e)}", ""

    answer = result.get("answer", "No answer found.")
    sources = result.get("sources", "")

    return answer, sources or "No sources available."

# 5. Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# RockyBot: News Research Tool (Groq + Gradio)")

    with gr.Row():
        url1 = gr.Textbox(label="URL 1")
        url2 = gr.Textbox(label=" URL 2")
        url3 = gr.Textbox(label="URL 3")
        url4 = gr.Textbox(label="URL 4")
        url5 = gr.Textbox(label="URL 5")
        url6 = gr.Textbox(label="URL 6")
        url7 = gr.Textbox(label="URL 7")
    process_btn = gr.Button("Process URLs")
    status_output = gr.Textbox(label="Status")

    process_btn.click(process_urls, [url1, url2, url3], status_output)

    question_input = gr.Textbox(label="Ask a Question")
    answer_output = gr.Textbox(label="Answer")
    sources_output = gr.Textbox(label="Sources")

    question_input.submit(answer_question, question_input, [answer_output, sources_output])

# Launch the app
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbdc9fb952c0b29232.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
pip install langchain-groq


In [ ]:
!pip install langchain langchain-community langchain-groq gradio faiss-cpu sentence-transformers unstructured requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 68.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━

In [5]:
import os
import pickle
import time
import gradio as gr

from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
userdata.get('News_API')


# 1. Set Groq API Key
os.environ["GROQ_API_KEY"] = "News_API"

# 2. Initialize LLM (Groq)
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-70b-8192",  # Groq supported model
    temperature=0.7
)

file_path = "faiss_store_groq.pkl"
vectorstore = None

# 3. Function to process URLs and build FAISS index
def process_urls(url1, url2, url3):
    urls = [u for u in [url1, url2, url3] if u.strip()]
    if not urls:
        return "Please provide at least one valid URL."

    loader = UnstructuredURLLoader(urls=urls)
    try:
        data = loader.load()
    except Exception as e:
        return f"Error loading URLs: {str(e)}"

    if not data:
        return "No content fetched from the provided URLs."

    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
    docs = text_splitter.split_documents(data)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore_groq = FAISS.from_documents(docs, embeddings)

    with open(file_path, "wb") as f:
        pickle.dump(vectorstore_groq, f)

    return "Processing completed! You can now ask questions."

def answer_question(question):
    if not os.path.exists(file_path):
        return "Please process URLs first!", ""

    with open(file_path, "rb") as f:
        vectorstore = pickle.load(f)

    chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

    try:
        result = chain({"question": question}, return_only_outputs=True)
    except Exception as e:
        return f"Error generating answer: {str(e)}", ""

    answer = result.get("answer", "No answer found.")
    sources = result.get("sources", "")

    return answer, sources or "No sources available."

# 5. Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# RockyBot: News Research Tool (Groq + Gradio)")

    with gr.Row():
        url1 = gr.Textbox(label="URL 1")
        url2 = gr.Textbox(label=" URL 2")
        url3 = gr.Textbox(label="URL 3")
    process_btn = gr.Button("Process URLs")
    status_output = gr.Textbox(label="Status")

    process_btn.click(process_urls, [url1, url2, url3], status_output)

    question_input = gr.Textbox(label="Ask a Question")
    answer_output = gr.Textbox(label="Answer")
    sources_output = gr.Textbox(label="Sources")

    question_input.submit(answer_question, question_input, [answer_output, sources_output])

# Launch the app
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://125ea3c6a0c22343a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
